In [3]:
import pandas as pd
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import bulk, streaming_bulk
import uuid
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}],timeout=30, max_retries=10, retry_on_timeout=True)
import json
import time
from pandas.io.json import json_normalize


In [18]:
df = pd.DataFrame()

start = time.time()
page = es.search(
    index='alldataindex',
    scroll='40m',
    size=10000,
    _source=["id","content"],
    body={},
    request_timeout=10000
)
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])

df = df.append(json_normalize(page['hits']['hits']))
i=1
while (scroll_size > 0):
    #print("scrolling..",i)
    i=i+1
    page = es.scroll(scroll_id = sid, scroll = '40m')
    df = df.append(json_normalize(page['hits']['hits']))
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])
end = time.time() - start
print(end)
print(len(df))


307.08069705963135
1000000


In [19]:
len(df)

1000000

In [22]:
df.head(2)

,_id,_index,_score,_source.content,_source.id,_type
0,C7kplWsBcyYnpedmfvCc,alldataindex,1.0,TWO THOUSAND... SIX HUNDRED AND SIXTY-SIX DOLL...,946b2ba4-832b-4501-b802-8b4955dcc129,alldataindex_type
1,DLkplWsBcyYnpedmfvCc,alldataindex,1.0,Unleashing your entrepreneurial spirit towards...,96e1f93e-3182-4754-b24a-0ad5bf2f6631,alldataindex_type


# preprocess function

In [20]:
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
import string as str
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('wordnet')
punctuation_signs=list('''!()-[]{};:'"\,<>./?#^_~''')
wordnet_lemmatizer = WordNetLemmatizer()
stop_words =list(stopwords.words('english')) + punctuation_signs
category_codes = {
    'Business': 0,
    'Entertainment': 1,
    'Politics': 2,
    'Sport': 3,
    'Tech': 4
}


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ssm392/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ssm392/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ssm392/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
import numpy as np 
import concurrent.futures
from functools import partial

def process_text(id1,val):
        #print('id:',id1,'val:',val)
        #print('----------------------')
        dict_parsed = {}
        #strval = pd.Series.to_string(val)
        res = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in nltk.word_tokenize(val.lower().replace("'s"," ")) if word.lower() not in stop_words]
        dict_parsed['NLPid'] = id1
        dict_parsed['Content_Parsed'] = ' '.join(res)
        dict_parsed['Category_code'] = " "
        dict_parsed['News_length'] = len(val)
        return dict_parsed 
    
    

# Swifter 

In [25]:
import swifter

In [68]:
df.head(1)

,_id,_index,_score,_source.content,_source.id,_type
0,C7kplWsBcyYnpedmfvCc,alldataindex,1.0,TWO THOUSAND... SIX HUNDRED AND SIXTY-SIX DOLL...,946b2ba4-832b-4501-b802-8b4955dcc129,alldataindex_type


In [70]:
type(df[['_source.id', '_source.content']])

pandas.core.frame.DataFrame

In [74]:
%time NLPdf = df[['_source.id', '_source.content']].swifter.apply(lambda row: process_text(row['_source.id'], row['_source.content']),axis=1)


CPU times: user 2h 3min 20s, sys: 2min 21s, total: 2h 5min 41s
Wall time: 4h 31min 23s


In [75]:
type(NLPdf)

pandas.core.series.Series

In [80]:
len(NLPdf)

1000000

In [82]:
ppdf = NLPdf.swifter.apply(pd.Series)

In [83]:
ppdf.head(5)

,NLPid,Content_Parsed,Category_code,News_length
0,946b2ba4-832b-4501-b802-8b4955dcc129,two thousand ... six hundred sixty-six dollaaa...,,754
1,96e1f93e-3182-4754-b24a-0ad5bf2f6631,unleash entrepreneurial spirit towards own tow...,,4178
2,977bff20-fde0-45d4-bdf7-c2e69e0928d0,'re one country leaders probably n't get much ...,,928
3,40ed5fcd-8cd9-47ab-b363-548866b76221,athlone gaa player owe life quick action medic...,,2429
4,685c537c-89dd-4e1b-96e6-e40275c8d9c4,emerson electric nyse emr open bullishly pivot...,,232


In [79]:
import pickle
with open(r'/Users/ssm392/Desktop/IV Project/data/ppdata.jsonl', 'wb') as output:
    pickle.dump(NLPdf, output)
     

In [44]:
len(df3)
#df=pd.DataFrame()

1000000

In [ ]:
## Load preprocessed data to ES

In [149]:
from elasticsearch import Elasticsearch 
from elasticsearch.helpers import bulk, streaming_bulk 

es = Elasticsearch([{'host': 'localhost', 'port': '9200'}]) 

use_these_keys = ['NLPid', 'Content_Parsed','Category_code','News_length'] 

def filterKeys(document): 
    return {key: document[key] for key in use_these_keys }

  

def bulk_json_data(_index, doc_type):  
    for index, document in ppdf.iterrows(): 
        yield { 
            "_index": _index, 
            "_type": doc_type, 
            "_source": filterKeys(document) 
        } 

success, failed = 0, 0 


# list of errors to be collected is not stats_only 

errors = [] 
i=1 

for ok, item in streaming_bulk(es, bulk_json_data('ppalldataindex','doc'), chunk_size=1000, max_retries=5,initial_backoff=2, max_backoff=600, request_timeout=3600): 

    # go through request-reponse pairs and detect failures 
    if not ok: 
        if not stats_only: 
            errors.append(item) 
        failed += 1 

    else: 
        success += 1 
        #print(success) 

print(success) 

1000000
